In [0]:
# !pip install -U ginza
# import pkg_resources, imp
# imp.reload(pkg_resources)

In [0]:
import spacy
nlp = spacy.load('ja_ginza')
doc = nlp('銀座でランチをご一緒しましょう。')
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.i)
    print('EOS')

0 銀座 銀座 PROPN 名詞-固有名詞-地名-一般 compound 5
1 で で ADP 助詞-格助詞 case 0
2 ランチ ランチ NOUN 名詞-普通名詞-一般 obj 5
3 を を ADP 助詞-格助詞 case 2
4 ご 御 NOUN 接頭辞 compound 5
5 一緒 一緒 VERB 名詞-普通名詞-サ変可能 ROOT 5
6 し 為る AUX 動詞-非自立可能 aux 5
7 ましょう ます AUX 助動詞 aux 5
8 。 。 PUNCT 補助記号-句点 punct 5
EOS


## NER の例

In [0]:
doc = nlp("ソフトバンクグループは30日、ことし３月期のグループ全体の業績見通しを下方修正し、" + 
          "最終赤字の額がこれまでの見込みより1500億円増え、9000億円の赤字になると発表しました。" + 
          "投資先のアメリカのシェアオフィス大手、ウィーワークの経営悪化によって支援策に関連する損失が当初の見込みより増えたことなどが理由です。")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

ソフトバンクグループ 0 10 Person
30日 11 14 Date
３月期 18 21 Date
1500億円 58 64 Money
9000億円 67 73 Money
アメリカ 91 95 Country
ウィーワーク 106 112 Company
経営悪化 113 117 Doctrine_Method_Other


ソフトバンクグループが人として認識されている．これをどうにかしたいので，モデルを学習する．汎化性能を見るためにテストデータも用意する．

In [0]:
text_train = "ソフトバンクグループは30日、ことし３月期のグループ全体の業績見通しを下方修正し、" + \
"最終赤字の額がこれまでの見込みより1500億円増え、9000億円の赤字になると発表しました。" + \
"投資先のアメリカのシェアオフィス大手、ウィーワークの経営悪化によって支援策に関連する損失が当初の見込みより増えたことなどが理由です。"
doc_train = nlp(text_train)

text_test = "去年３月期の決算では１兆4000億円余りの最終黒字を確保したソフトバンクグループは今回は一転して巨額の赤字となります。" + \
"年間の決算で最終赤字となるのは15年ぶりで赤字幅は過去最大です。"
# text_test = "ソフトバンクグループは、新型コロナウイルスの感染拡大を背景にした市場の動揺で" + \
#             "投資先の企業の価値が下がったことから、業績が急激に悪化しました。"
doc_test = nlp(text_test)

print("train")
for ent in doc_train.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print("\ntest")
for ent in doc_test.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

train
ソフトバンクグループ 0 10 Person
30日 11 14 Date
３月期 18 21 Date
1500億円 58 64 Money
9000億円 67 73 Money
アメリカ 91 95 Country
ウィーワーク 106 112 Company
経営悪化 113 117 Doctrine_Method_Other

test
去年３月期 0 5 Date
１兆4000億円余り 10 20 Money
15年ぶり 74 79 Period_Year


テストデータの `ソフトバンクグループ` は固有表現としても認識されていない．

訓練データの NER は `ソフトバンクグループ` 以外は正しいことにして，ソフトバンクグループの Person を Company に変えたものを正解データとして学習させる．[spaCy 101](https://spacy.io/usage/spacy-101#lightning-tour) にあるコードを流用．

In [0]:
correct_labels = [(ent.start_char, ent.end_char, ent.label_) for ent in doc_train.ents[1:]] # ソフトバンクグループ以外の固有表現はそのまま利用
print(corrrect_labels)
train_data = [(text_train, {"entities": [(0, 10, "Company")] + correct_labels})]

import random

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for i in range(10):
        random.shuffle(train_data)
        for text, annotations in train_data:
            nlp.update([text], [annotations], sgd=optimizer)

[(11, 14, 'Date'), (18, 21, 'Date'), (58, 64, 'Money'), (67, 73, 'Money'), (91, 95, 'Country'), (106, 112, 'Company'), (113, 117, 'Doctrine_Method_Other')]


/usr/lib/python3.6/runpy.py:193: UserWarning: [W019] Changing vectors name from ja_nopn.vectors to ja_nopn.vectors_153336, to avoid clash with previously loaded vectors. See Issue #3853.
  "__main__", mod_spec)


訓練データの `ソフトバンクグループ` が正しく Company で認識されるだけでなく，テストデータの `ソフトバンクグループ` もうまく行く

In [0]:
print("train")
for ent in nlp(text_train).ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
print("test")
for ent in nlp(text_test).ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

train
ソフトバンクグループ 0 10 Company
30日 11 14 Date
３月期 18 21 Date
1500億円 58 64 Money
9000億円 67 73 Money
アメリカ 91 95 Country
ウィーワーク 106 112 Company
経営悪化 113 117 Doctrine_Method_Other
test
３月期 2 5 Date
１兆4000億円余り 10 20 Money
ソフトバンクグループ 30 40 Company
15年ぶり 74 79 Date
